# Particionado

## Creamos una sesión de spark 

In [1]:
from pyspark.sql import SparkSession

Spark permite desde la creación de la sesión o contexto, indicar la cantidad de particiones que tendremos

Para esto debemos de indicar con '[ ]'  en la indicación de master la cantidad total de particiones

In [2]:
spark = SparkSession.builder.appName("Particionado").master( "local[5]").getOrCreate()

In [3]:
df = spark.range(0,20)
df.rdd.getNumPartitions()

5

El método 'parallelize', permite la asignar manualmente la cantidad de particiones.

In [4]:
rdd1 = spark.sparkContext.parallelize((0,20),6)
rdd1.getNumPartitions()

6

In [9]:
!ls /home/jaidenmeiden/development/study/spark/spark-python-testing/files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista.csv	 evento.csv	      paises.csv
deportista2.csv  juegos.csv	      resultados.csv


In [10]:
path = "files/"

Del mismo modo cuandore creamos un RDD o DF, podemos hacer esto.

En el caso de los RDD se realiza de la siguiente forma

In [11]:
rddDesdeArchivo = spark \
    .sparkContext \
    .textFile(path+"deporte.csv",10)

In [12]:
rddDesdeArchivo.getNumPartitions()

10

Es una buena practica tener los archivos de datos particionados para una carga mas rápida y mejor administración.

El método 'saveAsTextFile' permite almacenar los archivos, particionados o no, en un ruta.

In [13]:
rddDesdeArchivo.saveAsTextFile(path+"salidastexto")

In [15]:
!ls /home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/

_SUCCESS    part-00001	part-00003  part-00005	part-00007  part-00009
part-00000  part-00002	part-00004  part-00006	part-00008


In [17]:
# Visualizamos las primeras lineas de un archivo
!head -n 5 /home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/part-00001

7,Athletics
8,Ice Hockey
9,Swimming
10,Badminton
11,Sailing


A continuación se muestra como cargar los multiples archivos en un mismo RDD.

Esta operación tambien se puede realizar para DF

In [18]:
rdd = spark.sparkContext.wholeTextFiles(path+"salidastexto/*")

In [19]:
lista = rdd.mapValues(lambda x : x.split()).collect()

In [25]:
lista

[('file:/home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/part-00000',
  ['deporte_id,deporte',
   '1,Basketball',
   '2,Judo',
   '3,Football',
   '4,Tug-Of-War',
   '5,Speed',
   'Skating',
   '6,Cross',
   'Country',
   'Skiing']),
 ('file:/home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/part-00001',
  ['7,Athletics',
   '8,Ice',
   'Hockey',
   '9,Swimming',
   '10,Badminton',
   '11,Sailing',
   '12,Biathlon',
   '13,Gymnastics',
   '14,Art',
   'Competitions']),
 ('file:/home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/part-00002',
  ['15,Alpine',
   'Skiing',
   '16,Handball',
   '17,Weightlifting',
   '18,Wrestling',
   '19,Luge',
   '20,Water',
   'Polo']),
 ('file:/home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/part-00003',
  ['21,Hockey',
   '22,Rowing',
   '23,Bobsleigh',
   '24,Fencing',
   '25,Equestrianism',
   '26,Shooting',
   '27,Boxin

In [26]:
l = [l[0] for l in lista]
l.sort()

In [28]:
l

['file:/home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/part-00000',
 'file:/home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/part-00001',
 'file:/home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/part-00002',
 'file:/home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/part-00003',
 'file:/home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/part-00004',
 'file:/home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/part-00005',
 'file:/home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/part-00006',
 'file:/home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/part-00007',
 'file:/home/jaidenmeiden/development/study/spark/spark-python-testing/files/salidastexto/part-00008',
 'file:/home/jaidenmeiden/development/study/spark/spark-python-testing/fi

In [27]:
rddDesdeArchivo = spark \
    .sparkContext \
    .textFile(','.join(l),
              10).map(lambda l : l.split(","))


In [29]:
rddDesdeArchivo.take(7)

[['deporte_id', 'deporte'],
 ['1', 'Basketball'],
 ['2', 'Judo'],
 ['3', 'Football'],
 ['4', 'Tug-Of-War'],
 ['5', 'Speed Skating'],
 ['6', 'Cross Country Skiing']]

In [30]:
!pwd

/home/jaidenmeiden/development/study/spark/spark-python-testing


In [31]:
spark.stop()